In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()
# If you have built two spark sessions on local, the first can be accessed via 4040 and the second via 4041. 
spark

23/02/08 02:06:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import pandas as pd
from pyspark.sql import types

In [70]:
# Test functionality
df_green = pd.read_parquet('data/pq/green/2020/01/green_tripdata_2020_01.parquet')
df_green = df_green.drop(['ehail_fee'], axis=1)
df_green_head = df_green.head(10)
df_green_head
# Show the schema of a Spark df created from a pandas df
spark.createDataFrame(df_green_head)

DataFrame[VendorID: bigint, lpep_pickup_datetime: timestamp, lpep_dropoff_datetime: timestamp, store_and_fwd_flag: string, RatecodeID: double, PULocationID: bigint, DOLocationID: bigint, passenger_count: double, trip_distance: double, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, payment_type: double, trip_type: double, congestion_surcharge: double]

In [71]:
df_green_head

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2019-12-18 15:52:30,2019-12-18 15:54:39,N,1.0,264,264,5.0,0.00,3.5,0.50,0.5,0.01,0.0,0.3,4.81,1.0,1.0,0.00
1,2,2020-01-01 00:45:58,2020-01-01 00:56:39,N,5.0,66,65,2.0,1.28,20.0,0.00,0.0,4.06,0.0,0.3,24.36,1.0,2.0,0.00
2,2,2020-01-01 00:41:38,2020-01-01 00:52:49,N,1.0,181,228,1.0,2.47,10.5,0.50,0.5,3.54,0.0,0.3,15.34,1.0,1.0,0.00
3,1,2020-01-01 00:52:46,2020-01-01 01:14:21,N,1.0,129,263,2.0,6.30,21.0,3.25,0.5,0.00,0.0,0.3,25.05,2.0,1.0,2.75
4,1,2020-01-01 00:19:57,2020-01-01 00:30:56,N,1.0,210,150,1.0,2.30,10.0,0.50,0.5,0.00,0.0,0.3,11.30,1.0,1.0,0.00
5,1,2020-01-01 00:52:33,2020-01-01 01:09:54,N,1.0,35,39,1.0,3.00,13.5,0.50,0.5,0.00,0.0,0.3,14.80,1.0,1.0,0.00
6,2,2020-01-01 00:10:18,2020-01-01 00:22:16,N,1.0,25,61,1.0,2.77,11.0,0.50,0.5,0.00,0.0,0.3,12.30,2.0,1.0,0.00
7,2,2020-01-01 01:03:14,2020-01-01 01:29:45,N,1.0,225,89,1.0,4.98,20.5,0.50,0.5,0.00,0.0,0.3,21.80,2.0,1.0,0.00
8,2,2020-01-01 00:04:11,2020-01-01 00:09:48,N,1.0,129,129,1.0,0.71,5.5,0.50,0.5,0.00,0.0,0.3,6.80,2.0,1.0,0.00
9,2,2020-01-01 00:25:52,2020-01-01 00:32:16,N,1.0,129,83,1.0,0.80,5.5,0.50,0.5,0.00,0.0,0.3,6.80,2.0,1.0,0.00


In [ ]:
year = 2020
for month in range(1, 13):
    print(f'processing data for {year}/{month}')
    input_path = f'data/raw/green/{year}/{month:02d}/'
    output_path = f'data/pq/green/{year}/{month:02d}/'
    
    df_green = spark.read \
        .option("header", "true") \
        .parquet(input_path)
    
    df_green \
        .repartition(4) \
        .write.parquet(output_path)

In [ ]:
year = 2021
for month in range(1, 13):
    print(f'processing data for {year}/{month}')
    input_path = f'data/raw/green/{year}/{month:02d}/'
    output_path = f'data/pq/green/{year}/{month:02d}/'
    
    df_green = spark.read \
        .option("header", "true") \
        .parquet(input_path)
    
    df_green \
        .repartition(4) \
        .write.parquet(output_path)

In [79]:
# You can read partitioned parquet files by parsing the directory to speak.read.parquet
df = spark.read.parquet('data/pq/green/2021/01')
df \
    .select('lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .show()

+--------------------+---------------------+------------+------------+
|lpep_pickup_datetime|lpep_dropoff_datetime|PULocationID|DOLocationID|
+--------------------+---------------------+------------+------------+
| 2021-01-13 09:59:00|  2021-01-13 10:12:00|         169|         174|
| 2021-01-18 20:36:47|  2021-01-18 20:39:11|          42|         247|
| 2021-01-07 12:26:00|  2021-01-07 12:36:00|         224|         141|
| 2021-01-13 05:39:00|  2021-01-13 05:43:00|          61|          61|
| 2021-01-05 08:05:00|  2021-01-05 08:22:00|          80|          33|
| 2021-01-01 04:45:36|  2021-01-01 04:51:15|          74|          41|
| 2021-01-16 09:35:00|  2021-01-16 09:55:18|          41|         144|
| 2021-01-18 14:49:00|  2021-01-18 15:01:00|          72|          39|
| 2021-01-17 13:47:51|  2021-01-17 13:57:26|          95|          95|
| 2021-01-06 14:52:00|  2021-01-06 14:55:00|         169|         169|
| 2021-01-07 13:13:00|  2021-01-07 13:32:00|         244|          41|
| 2021

In [3]:
year = 2020
for month in range(1, 13):
    print(f'processing data for {year}/{month}')
    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'
    
    df_green = spark.read \
        .option("header", "true") \
        .parquet(input_path)
    
    df_green \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2020/1


processing data for 2020/2


processing data for 2020/3


processing data for 2020/4


processing data for 2020/5


...

processing data for 2020/8


processing data for 2020/9


processing data for 2020/10


processing data for 2020/11


processing data for 2020/12


In [4]:
year = 2021
for month in range(1, 13):
    print(f'processing data for {year}/{month}')
    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'
    
    df_green = spark.read \
        .option("header", "true") \
        .parquet(input_path)
    
    df_green \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2021/1


processing data for 2021/2


processing data for 2021/3


processing data for 2021/4


processing data for 2021/5


...

processing data for 2021/8


processing data for 2021/9


processing data for 2021/10


processing data for 2021/11


processing data for 2021/12
